In [21]:
!git clone https://github.com/leoxiaobin/deep-high-resolution-net.pytorch.git
%cd deep-high-resolution-net.pytorch




Cloning into 'deep-high-resolution-net.pytorch'...
remote: Enumerating objects: 296, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 296 (delta 0), reused 2 (delta 0), pack-reused 293 (from 1)
Receiving objects: 100% (296/296), 15.56 MiB | 33.40 MiB/s, done.
Resolving deltas: 100% (137/137), done.
/content/deep-high-resolution-net.pytorch/lib/deep-high-resolution-net.pytorch/lib/deep-high-resolution-net.pytorch/lib/deep-high-resolution-net.pytorch


In [9]:
!pip install -r requirements.txt --no-deps
!pip install opencv-python==4.5.5.64




  Preparing metadata (setup.py) ... done
ERROR: Ignored the following yanked versions: 3.4.11.39, 3.4.17.61, 4.4.0.42, 4.4.0.44, 4.5.4.58, 4.5.5.62, 4.7.0.68
ERROR: Could not find a version that satisfies the requirement opencv-python==3.4.1.15 (from versions: 3.4.0.14, 3.4.10.37, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.57, 3.4.16.59, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.60, 4.5.5.64, 4.6.0.66, 4.7.0.72, 4.8.0.74, 4.8.0.76, 4.8.1.78, 4.9.0.80, 4.10.0.82, 4.10.0.84)
ERROR: No matching distribution found for opencv-python==3.4.1.15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.10.0.84
    Uninstalling opencv-python-4.10.0.84:
      Successfully uninstalled opencv-python-4.10.0.84


In [25]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [26]:
%cd /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/lib
!make












/content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/lib
cd nms; python setup_linux.py build_ext --inplace; rm -rf build; cd ../../
running build_ext


In [27]:
# Clone the COCO API repository
!git clone https://github.com/cocodataset/cocoapi.git

# Install the Python API
%cd cocoapi/PythonAPI
!make install



Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975 (from 1)
Receiving objects: 100% (975/975), 11.72 MiB | 13.66 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/lib/cocoapi/PythonAPI
# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/lib/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x

In [33]:
# Download the pretrained model (example for HRNet W32)
!mkdir -p /content/drive/MyDrive/HRnet/models/pytorch/pose_coco
!wget -P /content/drive/MyDrive/HRnet/models/pytorch/pose_coco/ https://download.openmmlab.com/mmpose/hrnet/pose_hrnet_w32_256x192.pth

# Create output and log directories
%cd /content/drive/MyDrive/HRnet
!mkdir -p output log



--2024-08-26 19:03:33--  https://download.openmmlab.com/mmpose/hrnet/pose_hrnet_w32_256x192.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.48.231, 47.246.48.233, 47.246.48.239, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.48.231|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-08-26 19:03:35 ERROR 404: Not Found.

/content/drive/MyDrive/HRnet


In [35]:
!ls /content/drive/MyDrive/HRnet/models/pytorch/pose_coco/


pose_hrnet_w32_256x192.pth  pose_hrnet_w48_256x192.pth	pose_resnet_101_384x288.pth
pose_hrnet_w32_384x288.pth  pose_hrnet_w48_384x288.pth	pose_resnet_50_256x192.pth


In [11]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [36]:
import sys
sys.path.append('/content/deep-high-resolution-net.pytorch/lib')


In [38]:
!pip install yacs


In [110]:

import torch
from config import cfg, update_config
from core.inference import get_final_preds
from utils.transforms import get_affine_transform
from models.pose_hrnet import get_pose_net
from types import SimpleNamespace
import json

# Load configuration file
cfg_file = '/content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/experiments/coco/hrnet/w32_256x192_adam_lr1e-3.yaml'
args = SimpleNamespace(
    cfg=cfg_file,
    opts=[],
    modelDir=None,
    logDir=None,
    dataDir=None
)
update_config(cfg, args)

# Load the model architecture
model = get_pose_net(cfg, is_train=False)

# Load the state dictionary with weights
model_path = '/content/drive/MyDrive/HRnet/models/pytorch/pose_coco/pose_hrnet_w32_256x192.pth'
state_dict = torch.load(model_path, map_location=torch.device('cpu'))
model.load_state_dict(state_dict)
model.eval()

# Load and preprocess the image
image_path = '/content/drive/MyDrive/HRnet/IMG_8369.jpg'
image = cv2.imread(image_path)
input_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
c = np.array([input_img.shape[1] // 2, input_img.shape[0] // 2])
s = np.array([input_img.shape[1], input_img.shape[0]])
trans = get_affine_transform(c, s, 0, cfg.MODEL.IMAGE_SIZE)
model_input = cv2.warpAffine(input_img, trans, (int(cfg.MODEL.IMAGE_SIZE[0]), int(cfg.MODEL.IMAGE_SIZE[1])))
model_input = torch.from_numpy(model_input.transpose(2, 0, 1)).unsqueeze(0).float()

# Predict keypoints
with torch.no_grad():
    output = model(model_input)

# Transform to COCO format
output_np = output.cpu().numpy()
preds, maxvals = get_final_preds(cfg, output_np, [c], [s])
keypoints = preds[0].tolist()

# Prepare COCO-style JSON
coco_output = {
    "image_id": 1,
    "category_id": 1,
    "keypoints": [kp for kp in keypoints],
    "score": 1.0
}

# Save predictions
output_json = "/content/output_keypoints.json"
with open(output_json, 'w') as f:
    json.dump([coco_output], f)


<ipython-input-110-6815e9e423de>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=torch.device('cpu'))


Loading Model Visualization onto sample image

In [91]:


!python /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/demo.py \
    --image /content/drive/MyDrive/HRnet/IMG_8369.jpg \
    --showFps --write \
    --cfg /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/inference-config.yaml \
    TEST.MODEL_FILE /content/drive/MyDrive/HRnet/models/pytorch/pose_coco/pose_hrnet_w32_256x192.pth



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
=> loading model from /content/drive/MyDrive/HRnet/models/pytorch/pose_coco/pose_hrnet_w32_256x192.pth
/content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/demo.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to

In [95]:
import sys
sys.path.append('/content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/lib')


Loading Model Visualization onto Video

In [117]:
# !python /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/inference.py --cfg /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/inference-config.yaml \
#     --videoFile /content/drive/MyDrive/HRnet/IMG_4436.MOV \
#     --writeBoxFrames \
#     --outputDir output \
#     TEST.MODEL_FILE models/pytorch/pose_coco/pose_hrnet_w32_256x192.pth
!PYTHONPATH=/content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/lib \
python /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/inference.py \
    --cfg /content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/inference-config.yaml \
    --videoFile /content/drive/MyDrive/HRnet/swings/aaron_judge.mp4 \
    --writeBoxFrames \
    --outputDir /content/drive/MyDrive/HRnet/swing_pose_output \
    TEST.MODEL_FILE /content/drive/MyDrive/HRnet/models/pytorch/pose_coco/pose_hrnet_w32_256x192.pth


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
=> loading model from /content/drive/MyDrive/HRnet/models/pytorch/pose_coco/pose_hrnet_w32_256x192.pth
/content/drive/MyDrive/HRnet/models/deep-high-resolution-net.pytorch/demo/inference.py:206: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possib

In [122]:
!git clone https://github.com/ayushk1122/SwingSensei-AI-Model.git


Cloning into 'SwingSensei-AI-Model'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [123]:
%cd /content/drive/MyDrive/HRnet/SwingSensei-AI-Model

/content/drive/MyDrive/HRnet/SwingSensei-AI-Model


In [125]:
!git add /content/drive/MyDrive/swing_sensei_AI.ipynb


fatal: /content/drive/MyDrive/swing_sensei_AI.ipynb: '/content/drive/MyDrive/swing_sensei_AI.ipynb' is outside repository at '/content/drive/MyDrive/HRnet/SwingSensei-AI-Model'
